# Charts for my website

- Author: Kiril from Mindgraph
- Last meaningful update: 15-04-2025

This notebook creates some `plotly` charts for [my website](www.mindgraph.dk). The charts are displayed as visuals in this notebook and also exported to `.html` files on GitHub so that they can be directly embedded on the website.

In [54]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio

In [55]:
# Importing data
sales_data = pd.read_parquet("Output data/FinalSalesAndIncome.parquet")
indexed_development = pd.read_parquet("Output data/IndexedDevelopment.parquet")

# Getting min and max years for historical data
min_hist = sales_data[sales_data["PriceType"] == "Actual price"]["Year"].min()
max_hist = sales_data[sales_data["PriceType"] == "Actual price"]["Year"].max()

In [56]:
# # Changing to the default color palette in plotly
# pio.templates.default = "plotly"

In [57]:
# Creating a custom color palette with the MG colors
# Using the "Classy" palette from: https://mycolor.space/?hex=%231EA2B5&sub=1
my_custom_palette = ["#1ea2b5", "#324b4f", "#95b0b5", "#9f8ac3", "#6b588d"]
my_template = pio.templates["plotly_white"].layout.template
my_template.layout.colorway = my_custom_palette
pio.templates["my_custom_template"] = my_template
pio.templates.default = "my_custom_template"

## Template functions for chart

If we need to recreate the same chart with different subsamples of the data, it makes more sense to store the chart as a function:

In [58]:
def plot_price(
    sales_data: pd.DataFrame,
    metric_for_use: str,
    chart_title: str,
    selected_locations: list[str],
    price_types: list[str],
) -> None:

    # Preparing data for chart
    data_to_display = sales_data.copy()
    data_to_display = data_to_display[
        data_to_display["Municipality"].isin(selected_locations)
    ]
    data_to_display = data_to_display[
        data_to_display["PriceType"].isin(price_types)
    ].copy()

    # Rounding off to 2 decimals
    data_to_display[metric_for_use] = np.round(data_to_display[metric_for_use], 2)

    # Sorting and cleaning up
    data_to_display.sort_values("Year", ascending=False, inplace=True)
    data_to_display.reset_index(inplace=True, drop=True)

    # Identifying the selected municipalities
    unique_mncp = data_to_display["Municipality"].unique()

    # Checking whether multiple municipalities are selected
    if len(unique_mncp) > 1:
        color_var_for_chart = "Municipality"
    else:
        color_var_for_chart = None

    # Preparing text-based descriptions of the metrics
    if metric_for_use == "AvgSalesPrice":
        metric_for_use_txt = "Average price per m²"
    elif metric_for_use == "AvgSalesPriceIdx":
        metric_for_use_txt = "Average price per m² (indexed)"
    elif metric_for_use == "M2AffordedTotal":
        metric_for_use_txt = "m² buyable with annual disposable income"
    elif metric_for_use == "YearsoBuy50M2Total":
        metric_for_use_txt = "Years of annual income needed to buy 50 m²"
    else:
        metric_for_use_txt = "Unknown"

    # Plotting the data on a chart
    fig = px.line(
        data_to_display,
        x="Year",
        y=metric_for_use,
        color=color_var_for_chart,
        labels={
            "Year": "Year",
            metric_for_use: metric_for_use_txt,
        },
    )
    fig.update_layout({"title": chart_title})
    fig.update_traces(textposition="top center")
    fig
    return fig

## Figure 1

In [59]:
# Setting user parameters
n_mncp_to_show = 15
price_types = ["Actual price"]
chart_title = f"The {n_mncp_to_show} municipalities with the highest price increase between {min_hist}-{max_hist}"

In [60]:
# Filtering the data and sorting in the appropriate order
data_for_plot = sales_data.copy()
data_for_plot = data_for_plot[data_for_plot["Year"] == max_hist].copy()
data_for_plot = data_for_plot[
    data_for_plot["Municipality"] != "National average"
].copy()
data_for_plot.sort_values("AvgSalesPriceChange", ascending=False, inplace=True)
data_for_plot.reset_index(inplace=True, drop=True)
data_for_plot = data_for_plot[:n_mncp_to_show].copy()
data_for_plot.sort_values("AvgSalesPriceChange", inplace=True)

# Plotting the data on a chart
fig = px.bar(
    data_for_plot,
    x="AvgSalesPriceChange",
    y="Municipality",
    orientation="h",
    title=f"Top {n_mncp_to_show} municipalities in {max_hist} with the highest change in price per m² relative to 1992",
    labels={
        "Municipality": "Municipality",
        "AvgSalesPriceChange": "% change in average price per m²",
    },
)
fig

In [61]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure1.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [62]:
# Exporting as a PNG file (requires the "kaleido" package)
fig.write_image("Charts/Figure1.png")

## Figure 2

In [63]:
# Setting user parameters
n_mncp_to_show = 15
price_types = ["Actual price"]
chart_title = f"The {n_mncp_to_show} most expensive municipalities for buying a flat in {max_hist}"

In [64]:
# Filtering the data and sorting in the appropriate order
data_for_plot = sales_data[sales_data["Year"] == max_hist].copy()
data_for_plot = data_for_plot[
    data_for_plot["Municipality"] != "National average"
].copy()
data_for_plot.sort_values("AvgSalesPrice", ascending=False, inplace=True)
data_for_plot.reset_index(inplace=True, drop=True)
data_for_plot = data_for_plot[:n_mncp_to_show].copy()
data_for_plot.sort_values("AvgSalesPrice", inplace=True)

# Plotting the data on a chart
fig = px.bar(
    data_for_plot,
    x="AvgSalesPrice",
    y="Municipality",
    orientation="h",
    title=f"Top {n_mncp_to_show} most expensive municipalities in {max_hist}",
    labels={
        "Municipality": "Municipality",
        "AvgSalesPrice": "Average sales price per m²",
    },
)
fig

In [65]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure2.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [66]:
# Exporting as a PNG file
fig.write_image("Charts/Figure2.png")

## Figure 3

In [67]:
# Setting user parameters
price_types = ["Actual price"]
chart_title = (
    "Indexed sales price, national disposable income and national GDP over time"
)

In [68]:
# Preparing data for chart
data_to_display = indexed_development[
    indexed_development["Municipality"] == "National average"
]
data_to_display = data_to_display[data_to_display["PriceType"].isin(price_types)].copy()

# Sorting and cleaning up
data_to_display.sort_values("Year", ascending=False, inplace=True)
data_to_display.reset_index(inplace=True, drop=True)

# Rounding all relevant metrics off to 2 decimals
for metric in [
    "AvgSalesPriceIdx",
    "AvgDispIncomeIdx",
    "GDPIdx",
]:
    data_to_display[metric] = np.round(data_to_display[metric], 2)

# Converting the data to the long format before plotting
value_vars = ["AvgSalesPriceIdx", "AvgDispIncomeIdx", "GDPIdx"]
vars_mapping = {
    "AvgSalesPriceIdx": "Average sales price (indexed)",
    "AvgDispIncomeIdx": "Average disposable income (indexed)",
    "GDPIdx": "GDP (indexed)",
}
data_to_display = pd.melt(
    data_to_display,
    id_vars="Year",
    value_vars=value_vars,
    var_name="Indicator",
    value_name="Index",
)
data_to_display["Indicator"] = data_to_display["Indicator"].map(vars_mapping)

In [69]:
# Plotting the data on a chart
fig = px.line(
    data_to_display,
    x="Year",
    y="Index",
    color="Indicator",
)
fig.update_layout({"title": chart_title})
fig.update_traces(textposition="top center")
fig

In [70]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure3.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [71]:
# Exporting as a PNG file
fig.write_image("Charts/Figure3.png")

## Figure 4

In [72]:
# Creating the chart
selected_locations = ["National average"]
chart_title = (
    "Number of m² that can be bought with annual disposable income (national average)"
)
fig = plot_price(
    sales_data, "M2AffordedTotal", chart_title, selected_locations, ["Actual price"]
)
fig

In [73]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure4.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [74]:
# Exporting as a PNG file
fig.write_image("Charts/Figure4.png")

## Figure 5

In [75]:
# Creating the chart
selected_locations = ["København", "Frederiksberg", "Aarhus", "Odense", "Aalborg"]
chart_title = "Number of m² that can be bought with average annual disposable income in Denmark's four largest cities"
fig = plot_price(
    sales_data, "M2AffordedTotal", chart_title, selected_locations, ["Actual price"]
)
fig

In [76]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure5.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [77]:
# Exporting as a PNG file
fig.write_image("Charts/Figure5.png")

## Figure 6

In [78]:
# Creating the chart
selected_locations = ["Randers", "Brønderslev", "Faxe"]
chart_title = "Number of m² that can be bought with average annual disposable income in municipalities that do not follow the general trend"
fig = plot_price(
    sales_data, "M2AffordedTotal", chart_title, selected_locations, ["Actual price"]
)
fig

In [79]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure6.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [80]:
# Exporting as a PNG file
fig.write_image("Charts/Figure6.png")

## Figure 7

In [81]:
# Creating the chart
selected_locations = ["København", "Frederiksberg", "Aarhus", "Odense", "Aalborg"]
chart_title = "Average price per m² for Denmark's four largest cities over time (incl. predictions)"
fig = plot_price(
    sales_data,
    "AvgSalesPrice",
    chart_title,
    selected_locations,
    ["Actual price", "Predicted price"],
)
fig

In [82]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure7.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [83]:
# Exporting as a PNG file
fig.write_image("Charts/Figure7.png")

## Figure 8

In [84]:
# Creating the chart
selected_locations = ["København", "Frederiksberg", "Aarhus", "Odense", "Aalborg"]
chart_title = "Average price per m² for Denmark's four largest cities over time (incl. predictions)"
fig = plot_price(
    sales_data,
    "AvgSalesPrice",
    chart_title,
    selected_locations,
    ["Actual price"],
)
fig

In [85]:
# Export to a standalone HTML file
html_str = pio.to_html(fig, full_html=False, include_plotlyjs="cdn")
with open("Charts/Figure8.html", "w", encoding="utf-8") as f:
    f.write(html_str)

In [86]:
# Exporting as a PNG file
fig.write_image("Charts/Figure8.png")